## 15. How long is the wait to get the potato salad? 

Please refer to page 77-80 of [the book](https://books.google.ch/books?id=bmhuaGP3FOEC&printsec=frontcover&hl=de&source=gbs_ge_summary_r&cad=0#v=onepage&q&f=false) for the problem description.

In [1]:
import numpy as np

def deliStats(λ, μ, clerkcount) :
    # λ -> frequency of customer arrivals
    # μ -> frequency of customer serviced 
    # clerkcount -> number of clerks 
    clock = 0 # keeping track of time
    # flags to see whether clerks are busy
    clerkbusy1 = 0
    clerkbusy2 = 0
    # length of the queue
    queuelength = 0
    # remaining service times 
    servicetimeremaining1 = 0
    servicetimeremaining2 = 0
    # queue log
    queue = (np.zeros([2,200])).astype(int)
    # idle time
    clerkidletime1 = 0
    clerkidletime2 = 0
    # deli closing time = expected # of customers that will arive in a 30 hour period
    # conservative overestimate of the # of customers that the clerks have to serve
    closedeli = int(np.ceil(30*λ))
    # array of customer arrival times
    customerarrivaltime = (np.zeros(closedeli)).astype(int)
    # arrival of the 1st customer 
    # λ/3600 is the avg. # of customers arriving per second
    customerarrivaltime[0] = (np.ceil(np.random.exponential(3600/λ))).astype(int)
    # arrival times of next customers
    for i in range(1,closedeli) :
        customerarrivaltime[i] = customerarrivaltime[i-1] + (np.ceil(np.random.exponential(3600/λ))).astype(int)
    # array of service times
    customerservicetime = (np.ceil(np.random.exponential(scale=3600/μ, size=closedeli))).astype(int)
    # sum of total waiting times of all customers who have appeared till a given point of time
    # = time in the queue + service time
    totaltime = 0
    # max of values
    maxtotaltime = 0 
    # sum of lengths of the queue 
    totalqueuelength = 0
    # max of values
    maxqueuelength = 0
    # index to denote how many customers arrived before
    newcustomer = 0
    # number of clerks?
    
    if clerkcount == 1 :
        clerkbusy2 = 1
        servicetimeremaining2 = 10**10
    
    while clock < 36000 :
        #print(clerkbusy1,clerkbusy2)
        
        # a second has passed so the service time should decrease by 1
        if servicetimeremaining1 > 0 :
            servicetimeremaining1 -= 1
        if servicetimeremaining2 > 0 :
            servicetimeremaining2 -= 1
        
        # if at least one of the clerks are free and there are people in the queue
        if (clerkbusy1 == 0 or clerkbusy2 == 0) and queuelength > 0 :
            if clerkbusy1 == 0 and clerkbusy2 == 1:
            # the 2nd part of the condition is redundant 
            # but is done for future readablity 
                clerkbusy1 = 1
                servicetimeremaining1 = queue[1,0]  
            elif clerkbusy1 == 1 and clerkbusy2 == 0 :
                clerkbusy2 = 1
                servicetimeremaining2 = queue[1,0]
            # if both clerks are free then the customer goes to one of them randomly
            # Nahin's code does not do this part – unless this is done there is a bias
            # towards clerk1 i.e. all customers go to him/her even if clerk2 is also free
            elif clerkbusy1 == 0 and clerkbusy2 == 0 :
                choose = np.random.choice([1,2])
                if choose == 1 :
                    clerkbusy1 = 1
                    servicetimeremaining1 = queue[1,0]  
                elif choose == 2 :
                    clerkbusy2 = 1
                    servicetimeremaining2 = queue[1,0]
            totaltime += queue[0,0] + queue[1,0] # wait time in queue + service time
            if queue[0,0] + queue[1,0] > maxtotaltime :
                # finding maxtotaltime
                maxtotaltime = queue[0,0] + queue[1,0]
            for i in range(queuelength) :
                # bring the next customer ahead in the queue
                queue[0,i] = queue[0,i+1]
                queue[1,i] = queue[1,i+1]
            queuelength -= 1
        
        # if a new customer arrives and at least one clerk is free
        if (clock == customerarrivaltime[newcustomer]) and (clerkbusy1 == 0 or clerkbusy2 == 0) :
            if clerkbusy1 == 0 and clerkbusy2 == 1:
                clerkbusy1 = 1 
                servicetimeremaining1 = customerservicetime[newcustomer] 
            elif clerkbusy1 == 1 and clerkbusy2 == 0:
                clerkbusy2 == 1
                servicetimeremaining2 = customerservicetime[newcustomer] 
            # if both clerks are free then the customer goes to one of them randomly
            elif clerkbusy1 == 0 and clerkbusy2 == 0 :
                choose = np.random.choice([1,2])
                if choose == 1 :
                    clerkbusy1 = 1
                    servicetimeremaining1 = customerservicetime[newcustomer]
                elif choose == 2 :
                    clerkbusy2 = 1
                    servicetimeremaining2 = customerservicetime[newcustomer] 
            totaltime += customerservicetime[newcustomer]
            if customerservicetime[newcustomer] > maxtotaltime :
                maxtotaltime = customerservicetime[newcustomer] 
            newcustomer += 1

        # if both clerks are busy and a customer arrives, add them to the queue
        elif (clock == customerarrivaltime[newcustomer]) and (clerkbusy1 == 1 and clerkbusy2 == 1) :
            queuelength += 1
            queue[0,queuelength-1] = 1
            queue[1,queuelength-1] = customerservicetime[newcustomer]
            newcustomer += 1
        
        if clerkbusy1 == 0 :
            clerkidletime1 += 1
        if clerkbusy2 == 0 :
            clerkidletime2 += 1
        
        for i in range(queuelength) :
            queue[0,i] += 1
            
        if servicetimeremaining1 == 0 :
            clerkbusy1 = 0
        if servicetimeremaining2 == 0 :
            clerkbusy2 = 0
        
        totalqueuelength += queuelength
        
        if queuelength > maxqueuelength :
            maxqueuelength = queuelength
        
        clock += 1
    #return customerservicetime

    if clerkcount == 1 :
        clerkidletime2 = float("NaN")
    
    return [round(totaltime/(newcustomer - 1 - queuelength),2), maxtotaltime, round(totalqueuelength/clock,2), 
            maxqueuelength, round(100*clerkidletime1/clock,2), round(100*clerkidletime2/clock,2) ]

In [2]:
import pandas as pd

colNames = ["Avg. total time", "Max. total time", "Avg. queue length",
            "Max. queue length", "Clerk 1 idle %", "Clerk 2 idle %"]

dat1clerks1 = pd.DataFrame([deliStats(30,40,1) for i in range(5)], columns=colNames)
dat1clerks1

,Avg. total time,Max. total time,Avg. queue length,Max. queue length,Clerk 1 idle %,Clerk 2 idle %
0,583.60,1881,4.28,20,18.58,NaN
1,458.79,2107,2.95,18,22.88,NaN
2,387.08,1783,2.56,17,21.28,NaN
3,368.37,1331,2.16,12,22.71,NaN
4,356.43,1215,2.18,10,23.69,NaN


In [3]:
dat1clerks2 = pd.DataFrame([deliStats(30,40,2) for i in range(5)], columns=colNames)
dat1clerks2

,Avg. total time,Max. total time,Avg. queue length,Max. queue length,Clerk 1 idle %,Clerk 2 idle %
0,90.59,411,0.03,3,69.37,78.62
1,96.15,526,0.08,5,64.49,74.79
2,100.40,548,0.06,3,65.72,77.22
3,99.03,516,0.05,5,66.42,81.18
4,97.96,580,0.05,3,62.86,76.65


In [4]:
dat2clerks1 = pd.DataFrame([deliStats(30,25,1) for i in range(5)], columns=colNames)
dat2clerks1

,Avg. total time,Max. total time,Avg. queue length,Max. queue length,Clerk 1 idle %,Clerk 2 idle %
0,2402.20,4237,16.70,31,0.99,NaN
1,5076.13,10467,40.76,69,1.01,NaN
2,4492.98,9931,39.25,85,1.22,NaN
3,2957.20,6794,24.80,60,0.86,NaN
4,3220.23,5249,26.09,45,0.16,NaN


In [5]:
dat2clerks2 = pd.DataFrame([deliStats(30,25,2) for i in range(5)], columns=colNames)
dat2clerks2

,Avg. total time,Max. total time,Avg. queue length,Max. queue length,Clerk 1 idle %,Clerk 2 idle %
0,199.88,824,0.44,8,51.76,59.31
1,226.41,1624,0.67,12,49.80,62.08
2,230.43,1221,0.62,11,39.68,60.22
3,150.81,861,0.13,6,51.39,75.14
4,175.34,657,0.30,6,45.59,61.59


The values obtained in the above tables are in the same ballpark as those in 182-183 of the book.